In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

rental_incomeDF = pd.read_csv('output_data/rental(total)_income_merged.csv')
rental_incomeDF


#Data Formatting 
rental_incomeDF['2015 index'] = (rental_incomeDF['2015 Yearly Rent']/rental_incomeDF['2015 Income']*100).map("{:.1f}".format)
rental_incomeDF['2016 index'] = (rental_incomeDF['2016 Yearly Rent']/rental_incomeDF['2016 Income']*100).map("{:.1f}".format)
rental_incomeDF['2017 index'] = (rental_incomeDF['2017 Yearly Rent']/rental_incomeDF['2017 Income']*100).map("{:.1f}".format)
rental_incomeDF['2015 Yearly Rent']= rental_incomeDF['2015 Yearly Rent'].map("${:,.0f}".format)
rental_incomeDF['2016 Yearly Rent']= rental_incomeDF['2016 Yearly Rent'].map("${:,.0f}".format)
rental_incomeDF['2017 Yearly Rent']= rental_incomeDF['2017 Yearly Rent'].map("${:,.0f}".format)
rental_incomeDF['2015 Income']= rental_incomeDF['2015 Income'].map("${:,.0f}".format)
rental_incomeDF['2016 Income']= rental_incomeDF['2016 Income'].map("${:,.0f}".format)
rental_incomeDF['2017 Income']= rental_incomeDF['2017 Income'].map("${:,.0f}".format)
rental_incomeDF =rental_incomeDF.sort_values('2017 index').reset_index(drop=True)
rental_incomeDF.tail()


,State,2015 Yearly Rent,2016 Yearly Rent,2017 Yearly Rent,'15-'16 Rent Diff.,'16-'17 Rent Diff.,2015 Income,2016 Income,2017 Income,'15-'16 Income Diff.,'16-'17 Income Diff.,2015 index,2016 index,2017 index
43,Florida,"$19,800","$20,280","$20,700",480.0,420.0,"$50,518","$52,277","$53,681",1759,1404,39.2,38.8,38.6
44,Maine,"$18,000","$17,100","$20,100",-900.0,3000.0,"$52,516","$51,950","$51,664",-566,-286,34.3,32.9,38.9
45,Massachusetts,"$29,100","$30,000","$30,000",900.0,0.0,"$70,214","$73,820","$73,227",3606,-593,41.4,40.6,41.0
46,California,"$27,510","$28,740","$31,500",1230.0,2760.0,"$65,843","$68,070","$69,759",2227,1689,41.8,42.2,45.2
47,New York,"$37,200","$36,570","$38,700",-630.0,2130.0,"$60,017","$62,758","$62,447",2741,-311,62.0,58.3,62.0


In [2]:
income = (input("What is your yearly[2017] income? " ))
input_rent = int(income)*0.3
print(f"You can afford a rent up to {input_rent} annually or {input_rent/12} monthly.")

What is your yearly[2017] income? 50000
You can afford a rent up to 15000.0 annually or 1250.0 monthly.


In [3]:
affordable_rentDF = rental_incomeDF.loc[rental_incomeDF['2017 index'].astype(float)<=30.0].sort_values('2017 index')
affordable_rentDF = rental_incomeDF[['State','2017 Yearly Rent', '2017 Income', '2017 index']].reset_index(drop=True)
affordable_rentDF

,State,2017 Yearly Rent,2017 Income,2017 index
0,Alabama,"$11,970","$51,113",23.4
1,Alaska,"$19,170","$72,231",26.5
2,Arizona,"$16,200","$61,125",26.5
3,Arkansas,"$12,600","$48,829",25.8
4,California,"$31,500","$69,759",45.2
5,Colorado,"$21,600","$74,172",29.1
6,Connecticut,"$21,000","$72,780",28.9
7,Delaware,"$15,600","$62,318",25.0
8,District of Columbia,"$31,200","$83,382",37.4
9,Florida,"$20,700","$53,681",38.6


In [11]:
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
import geopandas as gpd
from bokeh.plotting import figure, output_file, show

output_file("Affordable Rent 2017.html")




contiguous_usa = gpd.read_file('cb_2018_us_state_20m/cb_2018_us_state_20m.shp')
contiguous_usa.columns = ['STATEFP', 'STATENS', 'AFFGEOID', 'GEOID', 'STUSPS', 'State', 'LSAD',
       'ALAND', 'AWATER', 'geometry']
contiguous_usa.head()

index2017 = pd.merge(contiguous_usa, rental_incomeDF, on="State", suffixes=("_shape", ""))
index2017 = index2017.loc[~index2017['State'].isin(['Nebraska','Hawaii', 'Alaska'])]
index2017

geosource_index2017 = GeoJSONDataSource(geojson = index2017.to_json())

## Plot change in median hh income 2016 to 2017
# Define color palettes
palette = brewer['BuPu'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 20, high = 62) #make sure you set the low and high, if you forget it won't map
# Define custom tick labels for color bar.
tick_labels = {'20': '20','25': '25', '30': '30',
               '35':'35', '40':'40','45':'45', 
               '50':'50', '55':'55', '60':'60+'}

# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 300, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p_2017index = figure(title = 'Rental Price / Median Income Index (Affordable Rent =< 30 Index)', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p_2017index.xgrid.grid_line_color = None
p_2017index.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p_2017index.patches('xs','ys', source = geosource_index2017, #your geojson file
                   fill_color = {'field' :'2017 index',
                                 'transform' : color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)

# Create hover tool
p_2017index.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@State'),
                               ('Income Change','@{2017 index}{int}')])) #if you add {int} then it writes as integer, otherwise scientific notation
                                                                #the @ shows the column name, so here I am saying "in hover box show "Median Rent: [value in 'Value' column]{in integer format}
# Specify layout
p_2017index.add_layout(color_bar, 'below')

show(p_2017index)